# Python Decorators

### Mike Driscoll (@driscollis)



* https://github.com/driscollis/ or
* http://bit.ly/pythondecor

## About Me

 - Programming in Python for 10+ years
 - Blogs about Python at www.MouseVsPython.com
 - Writes books about Python

## Books

 - Python 101
 - Python 201: Intermediate Python
 - wxPython Recipes
 - Python Interviews
 - ReportLab: PDF Processing with Python (releases June 2018)

# Let's get started!

## Decorator

**Definition:** *A decorator will take the function they are decorating and extend its behavior while not actually modifying what the function itself does.*

# But first...

# The Humble Function

In [ ]:
def doubler(number):
    return number * 2

doubler(5)

# Function Parameters

 - normal
 - keyword
 - variable arguments
 - variable keyword arguments

In [ ]:
# normal

def params(x, y):
    print(x, y)
    
params(7, 1)

In [ ]:
# normal + keyword arguments

def params(x, y, z=2):
    print('args => ', x, y)
    print('keyword args => ', z)
    
params(7, 1)
params(10, 5, z='foo')

In [ ]:
# arbitrary number of arguments

def params(*args):
    print('args => ', args)
    
params(7, 1)

In [ ]:
# *args + a keyword argument

def params(*args, z=10):
    print('args => ', args)
    print('keyword args => ', z)
    
params(7, 1)
params(10, 5, z='foo')

In [ ]:
# artitrary number of args and kwargs

def params(*args, **kwargs):
    print('args => ', args)
    print('keyword args => ', kwargs)
    
args = [1, 2, 3]
kwargs = {'fizz': 10, 'buzz': 15}
params(*args, **kwargs)
params(*args, foo=5, bar=20)

# Everything in Python is an Object

# Functions are Objects Too

* You can pass normal types (ints, strs) 
* You can pass a function to another function. 

In [ ]:
def foo():
    return 'fizz'

def doubler(func):
    return func() * 2

print(doubler)
print(type(doubler))
doubler(foo)

# Functions have attributes

In [ ]:
def doubler(number):
    return number * 2

dir(doubler)

# Magic methods in Python

`__doc__` and `__name__` are called **magic methods** or **dunder**

 - These methods are special
 - Used for creating everything OOP

# Accessing Function Attributes

In [ ]:
def doubler(number):
    return number * 2

print(doubler.__doc__)
print(doubler.__name__)

If the function doesn't have a docstring, then `__doc__` returns `None`. Let's fix that:

In [ ]:
def doubler(number):
    """Doubles the number passed to it"""
    return number * 2 
    
print(doubler.__doc__)

## Asking a Function about itself inside the function

In [ ]:
# You can ask a function about itself inside the function
def doubler(x):
    name = doubler.__name__
    print('function name is "{name}"'.format(name=name))
    return x * 2

doubler(2)

## Adding new attibutes to functions

In [ ]:
# You can also attach data to the function
def my_function():
    # "my_function" is in a global scope inside the function block
    my_function.called = True
    
my_function.called = False
if my_function.called:
    print('This function was called already')
else:
    print('This function has never been called')

In [ ]:
my_function()

if my_function.called:
    print('This function was called already')
else:
    print('This function has never been called')

# Functions Can be Nested

In [ ]:
def doubler():
    def double(x):
        return x * 2
    return double

print(doubler())
print(doubler()(2))
d = doubler()
d(4)

In [ ]:
x = 10 # these can be accessed inside the functions
y = 20
def wrapper():
    def nested():
        x = 5  # shadows x outside
        print('Nested x: {} y: {}'.format(x, y))
    nested()
    print('Outside x: {} y: {}'.format(x, y)) 
wrapper()

# Which brings us to Closures!

A closure is simply a function that is returned by another function

In [ ]:
def multiply_by(x):
    """Function generator or factory"""
    def multiplier(num):
        """This is a closure"""
        return num * x
    return multiplier

print(multiply_by)
multiple_by_5 = multiply_by(5)
print(multiple_by_5)
print(multiple_by_5(6))

# Closure Uses

 - Function generators
 - Keep a common interface (the adapter pattern)
 - Eliminates code duplication (DRY)
 - Delays execution of a function

# Closures enable the creation of Decorators

**Definition:** *A decorator will take the function they are decorating and extend its behavior while not actually modifying what the function itself does.*

## Decorators are usually applied to 

 - functions
 - methods

In [ ]:
def info(func): # <-- Accepts a function
    def wrapper():
        print('start of wrapper')
        result = func() # <-- Executes the passed in function
        print('end of wrapper')
        return result
    return wrapper # <-- Returns a different function

def hello():
    return 'Hello'


print(hello.__name__)
hello = info(hello)
print(hello.__name__)
hello()    

# Using Decorator Syntax

Starting in Python 2.4 (circa 2004), you can use "@" to apply a decorator

In [ ]:
@info
def hello():
    return 'Hello'

hello() 
hello.__name__

# Real Functions have Arguments

In [ ]:
def info(func): 
    def wrapper():
        print('start of wrapper')
        result = func()
        print('end of wrapper')
        return result
    return wrapper 

@info
def hello(name):  # < -- This won't work
    return 'Hello {}'.format(name)


hello('Mike') 

In [ ]:
def info(func): 
    def wrapper(*args, **kwargs):  # <-- Need to pass in arguments 
        print('start of wrapper')
        result = func(*args, **kwargs) # <-- Use args / kwargs 
        print('end of wrapper')
        return result
    return wrapper 

@info
def hello(name):
    return 'Hello {}'.format(name)

hello('Mike')

# Stacking / Chaining Decorators

You can also apply multiple decorators to a single function

In [ ]:
def bold(func):
    def wrapper(*arg, **kwargs):
        return "<b>" + func(*arg, **kwargs) + "</b>"
    return wrapper

def italic(func):
    def wrapper(*arg, **kwargs):
        return "<i>" + func(*arg, **kwargs) + "</i>"
    return wrapper

@bold
@italic
def formatted_text():
    return 'Python rocks!'

formatted_text()

# Normal Decorators Cannot be Called

In [ ]:
def bold(func):
    # We need *args passed to wrapper()
    def wrapper(*args):                      
        # We need to add *args here too
        return "<b>" + func(*args) + "</b>"  
    return wrapper

@bold # <-- This does not work!
def formatted_text(text):
    return text

print(formatted_text('Python Rocks!'))

# Create a decorator that accepts arguments

In [ ]:
def my_password_decorator(trunc):
    '''Decorator Generator'''
    print('Decorator arg = {arg}'.format(arg=trunc))
    
    def the_real_decorator(function):
        
        def wrapper(*args, **kwargs):  # <-- The args/kwargs for the decorated function
            print('Function {} args: {} kwargs: {}'.format(
                function.__name__, str(args), str(kwargs)))
            return function(*args, **kwargs)[:trunc] # <-- truncate the password
        
        return wrapper
    
    return the_real_decorator

In [ ]:
@my_password_decorator(8)
def create_password(password):
    return password

create_password('long_P@$$w0Rd')

# Decorator Obfuscation

Decorators actually obfuscate the name of the function and the docstring

## Yes, it matters
 
 - Object pickling (serialization) requires the `__name__` be updated
 - `__doc__` should be updated for introspection

In [ ]:
def bold(func):
    def wrapper(*args):
        return "<b>" + func(*args) + "</b>"
    return wrapper

@bold
def formatted_text(text):
    """
    Format the passed in text
    """
    return text

print(formatted_text.__name__)
print(formatted_text.__doc__)

## You can fix this yourself

In [ ]:
def bold(func):
    def wrapper(*args):
        return "<b>" + func(*args) + "</b>"
    wrapper.__doc__ = func.__doc__
    wrapper.__name__ = func.__name__
    return wrapper

@bold
def formatted_text(text):
    """
    Format the passed in text
    """
    return text

print(formatted_text.__name__)
print(formatted_text.__doc__)

## But Python provides a way to fix this

In [ ]:
from functools import wraps

def bold(func):
    @wraps(func)
    def wrapper(*args):
        return "<b>" + func(*args) + "</b>"
    return wrapper

In [ ]:
@bold
def formatted_text(text):
    """
    Format the passed in text
    """
    return text

print(formatted_text.__name__)
print(formatted_text.__doc__)

# Class Decorators

Using a class as a decorator

In [ ]:
import functools

class MyDecorator:
    
    def __call__(self, func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            return func(*args, **kwargs)
        return wrapper
    
@MyDecorator  # This won't work
def adder(x, y):
    return x + y

adder(5, 6)

## When using classes for decorators, you need to create an instance

In [ ]:
import functools

class MyDecorator:
    def __call__(self, func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            return func(*args, **kwargs)
        return wrapper
    
decorator = MyDecorator()

@decorator
def adder(x, y):
    return x + y

adder(5, 6)

# Adding arguments with a Class decorator

In [ ]:
class PasswordTruncator:
    
    def __init__(self, trunc_length):
        print('in __init__')
        self.trunc_length = trunc_length
        print('Decorator args: {}'.format(self.trunc_length))
        
    def __call__(self, func):
        print('in __call__')
        @functools.wraps(func)
        def wrapped(*args, **kwargs):
            print('in wrapped()')
            return func(*args, **kwargs)[:self.trunc_length]
        return wrapped

In [ ]:
@PasswordTruncator(8)
def create_password(password):
    return password

create_password('long_P@$$w0Rd')

# Decorating a Class

Adding a decorator to a class without modifying behavior

In [ ]:
class MyActualClass:
    def __init__(self):
        print('in MyActualClass __init__()')
 
    def quad(self, value):
        return value * 4
 
obj = MyActualClass()
print(obj.quad(4))

In [ ]:
def decorator(cls):
    class Wrapper(cls):
        def doubler(self, value):
            return value * 2
    return Wrapper

In [ ]:
@decorator
class MyActualClass:
    def __init__(self):
        print('in MyActualClass __init__()')
 
    def quad(self, value):
        return value * 4
 
obj = MyActualClass()
print(obj.quad(4))
print(obj.doubler(5))

Or just use a subclass

In [ ]:
class MyActualClass:
    def __init__(self):
        print('in MyActualClass __init__()')

    def quad(self, value):
        return value * 4

In [ ]:
class MySubClass(MyActualClass):
    def doubler(self, value):
        return value * 2

obj = MySubClass()
print(obj.quad(4))
print(obj.doubler(5))

# Practical Decorators

You can use decorators for many things. Some of the most popular uses including creating decorators for authentication (django / flask) and logging.

## A Logging Decorator

In [ ]:
import logging

def log(func):
    """
    Log what function is called
    """
    def wrap_log(*args, **kwargs):
        name = func.__name__
        logger = logging.getLogger(name)
        logger.setLevel(logging.INFO)

        # add file handler
        fh = logging.FileHandler("/home/mdriscoll/Desktop/%s.log" % name)
        fmt = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
        formatter = logging.Formatter(fmt)
        fh.setFormatter(formatter)
        logger.addHandler(fh)

        logger.info("Running function: %s" % name)
        result = func(*args, **kwargs)
        logger.info("Result: %s" % result)
        return func
    return wrap_log

In [ ]:
@log
def doubler(number):
    """Doubles the number passed to it"""
    return number * 2

## Exercise(s)

1) How can we tell our logger decorator where to log? 
2) Can you figure out how to make the logger decorator log to stdout?


## An Example from Flask

http://flask.pocoo.org/docs/0.12/quickstart/#a-minimal-application

In [ ]:
from flask import Flask
app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'

# Other common decorator uses

 - caching
 - retrying a function that might fail
 - profiling
 - redirecting stdout / stderr

# Python's built-in decorators

 - @property
 - @staticmethod
 - @classmethod

# @property

In [68]:
# We don't need a setter or getter 
# here since we aren't doing any
# kind of checking on x

class Person:
    
    def __init__(self, age):
        self.age = age

In [70]:
e = Person(4)
print(e.age)
e.age = 10
print(e.age)

4
10


# There are limits to age

In [82]:
class Person:
    def __init__(self, age):
        self._age = age
    
    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, age):
        if age < 0:
            raise RuntimeError(f'Invalid age {age}')
        elif age > 115:
            raise RuntimeError(f'Invalid age {age}')
        else:
            self._age = age

In [85]:
p = Person(10)
p.age = 10
print(p.age)

10


# Decorators from the Standard Library

- @contextlib.contextmanager
- @functools.lru_cache
- @functools.total_ordering
- @functools.wraps

# Questions?



![title](Evaluation.PNG)